In [1]:
import pandas as pd
import collections
import pickle

In [2]:
def get_pull(date):
    shows = pd.read_pickle('pulls/pull_%s/shows_%s.pickle' % (date,  date))
    songs = pd.read_pickle('pulls/pull_%s/songs_%s.pickle' % (date,  date))
    live_songs = pd.read_pickle('pulls/pull_%s/live_songs_%s.pickle' % (date,  date))
    return shows, songs, live_songs

In [3]:
shows, songs, live_songs = get_pull('2020-12-18')